In [1]:
import pandas as pd
import duckdb as db

In [ ]:
df = pd.read_csv(r"C:\Users\USER\Desktop\car_sales_analysis\dataset\car_sales_records.csv")

### **General Calculation 1 (Monthly)**

This query will calculate the Total sales, profit and units sold by month.

The results can be used to determine:

1. Total sales, profit and units sold for the entire year

In [6]:
gen_calc_1 = db.sql(
"""
with months as (
    select
        unnest([1,2,3,4,5,6,7,8,9,10,11,12]) as date_month,
        unnest(['January','February','March','April',
                'May','June','July','August',
                'September','October','November','December']) as date_month_name
),
    monthly_sales as (
    select
        date_month,
        date_month_name,        
        sum(qty_sold)::int as total_units_sold,
        sum(total_price_usd) as total_sales,
        sum(total_profit_usd) as total_profit
    from df
    group by 1,2
    order by date_month asc
)
    select
        a.date_month,
        a.date_month_name,        
        coalesce(b.total_units_sold,0)::int as total_units_sold,
        coalesce(b.total_sales,0) as total_sales,
        coalesce(b.total_profit,0) as total_profit
    from months as a
    left join monthly_sales as b
    on a.date_month = b.date_month
    order by a.date_month asc
"""
).df()
gen_calc_1

,date_month,date_month_name,total_units_sold,total_sales,total_profit
0,1,January,2,53000.0,5300.0
1,2,February,3,48000.0,4800.0
2,3,March,30,743000.0,74300.0
3,4,April,35,772000.0,95400.0
4,5,May,33,782000.0,95700.0
5,6,June,36,715000.0,91500.0
6,7,July,34,772000.0,89400.0
7,8,August,35,751000.0,88700.0
8,9,September,36,882000.0,113700.0
9,10,October,34,792000.0,85800.0


### **General Calculation 2(Day of Week)**

This query will calculate the Total sales, profit and units sold by day of week(Sun-Sat).

The results can be used to determine:

1. Daily sales, profit and total units sold.

In [7]:
gen_calc_2 = db.sql(
    """
with dow_names as (
    select 
        unnest([0,1,2,3,4,5,6]) AS date_dow,
        unnest(['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']) as date_dow_name
),
    dow_sales as (
        select
            date_dow,
            date_dow_name,
            sum(qty_sold)::int as total_units_sold,
            sum(total_price_usd) as total_sales,
            sum(total_profit_usd) as total_profit
        from df
        group by 1,2
)
    select
        a.date_dow,
        a.date_dow_name,
        coalesce(b.total_units_sold,0) as total_units_sold,
        coalesce(b.total_sales,0) as total_sales,
        coalesce(b.total_profit,0) as total_profit
    from dow_names as a
    left join dow_sales as b
        on a.date_dow = b.date_dow
    order by a.date_dow asc
    """
).df()
gen_calc_2

,date_dow,date_dow_name,total_units_sold,total_sales,total_profit
0,0,Sunday,126,2726000.0,342800.0
1,1,Monday,45,1040000.0,120400.0
2,2,Tuesday,64,1483000.0,165100.0
3,3,Wednesday,0,0.0,0.0
4,4,Thursday,1,20000.0,2000.0
5,5,Friday,67,1576000.0,167800.0
6,6,Saturday,44,1046000.0,114400.0


### **General Calculation 3(Date)**

In [8]:
gen_calc_3 = db.sql(
    """
    select
        date,
        sum(qty_sold)::int as total_units_sold,
        sum(total_price_usd) as total_sales,
        sum(total_profit_usd) as total_profit
    from df
    group by 1
    order by date asc
    """
).df()

In [9]:
gen_calc_3

,date,total_units_sold,total_sales,total_profit
0,2024-01-05,1,25000.0,2500.0
1,2024-01-08,1,28000.0,2800.0
2,2024-02-12,2,28000.0,2800.0
3,2024-02-22,1,20000.0,2000.0
4,2024-03-01,3,96000.0,9600.0
...,...,...,...,...
213,2024-12-23,1,24000.0,2400.0
214,2024-12-24,2,48000.0,4800.0
215,2024-12-27,1,14000.0,1400.0
216,2024-12-28,1,25000.0,2500.0


### **Dealers Calculations**

**dealers_gen_calc_1**

calculation of total sales, profit, units sold per month

In [29]:
"""
Dealers General Calculation 1:
Monthly Sales per dealer

"""

dealers_gen_calc_1 = db.sql(
"""
with months as (
    select
        unnest([1,2,3,4,5,6,7,8,9,10,11,12]) as date_month,
        unnest([
            'January','February','March','April',
            'May','June','July','August',
            'September','October','November','December'
        ]) as date_month_name
),
dealers as (
    select
        distinct(dealer_name) as dealer_name
    from df
),
 dealer_months_combination as (
     select
        a.dealer_name,
        b.date_month,
        b.date_month_name
     from dealers as a 
     cross join months as b
     order by a.dealer_name asc, b.date_month asc
 ),
dealer_sales as (
    select
        dealer_name,
        date_month,
        date_month_name,
        sum(qty_sold) as total_units_sold,
        sum(total_price_usd) as total_sales,
        sum(total_profit_usd) as total_profit
    from df
    group by 1,2,3
)

select
    a.dealer_name,
    a.date_month,
    a.date_month_name,
    coalesce(b.total_units_sold, 0)::int as total_units_sold,
    coalesce(b.total_sales, 0)::float as total_sales,
    coalesce(b.total_profit, 0)::float as total_profit
from dealer_months_combination as a
left join dealer_sales as b on
    a.dealer_name = b.dealer_name
    and a.date_month = b.date_month
order by a.dealer_name asc, a.date_month asc
"""
).df()
dealers_gen_calc_1

,dealer_name,date_month,date_month_name,total_units_sold,total_sales,total_profit
0,AutoHub Cars,1,January,0,0.0,0.0
1,AutoHub Cars,2,February,0,0.0,0.0
2,AutoHub Cars,3,March,4,76000.0,7600.0
3,AutoHub Cars,4,April,6,124000.0,17200.0
4,AutoHub Cars,5,May,1,34000.0,3400.0
...,...,...,...,...,...,...
115,Visayas Vehicles,8,August,7,186000.0,18600.0
116,Visayas Vehicles,9,September,6,141000.0,28200.0
117,Visayas Vehicles,10,October,4,74000.0,7400.0
118,Visayas Vehicles,11,November,3,64000.0,6400.0


**dealers_gen_calc_2**

calculation of total sales, profit, units sold per day of week

In [11]:
"""
Dealers General Calculation 2:
Day of week Sales per dealer

"""

dealers_gen_calc_2 = db.sql(
"""
with dow_names as (
    select
        unnest([0,1,2,3,4,5,6]) as date_dow,
        unnest([
            'Sunday','Monday','Tuesday','Wednesday',
            'Thursday','Friday','Saturday'
        ]) as date_dow_name
),
dealers as (
    select
        distinct(dealer_name) as dealer_name
    from df
),
 dealers_dow_combination as (
     select
        a.dealer_name,
        b.date_dow,
        b.date_dow_name
     from dealers as a 
     cross join dow_names as b
     order by a.dealer_name asc, b.date_dow asc
 ),
dealer_sales as (
    select
        dealer_name,
        date_dow,
        date_dow_name,
        sum(qty_sold) as total_units_sold,
        sum(total_price_usd) as total_sales,
        sum(total_profit_usd) as total_profit
    from df
    group by 1,2,3
)

select
    a.dealer_name,
    a.date_dow,
    a.date_dow_name,
    coalesce(b.total_units_sold, 0)::int as total_units_sold,
    coalesce(b.total_sales, 0)::float as total_sales,
    coalesce(b.total_profit, 0)::float as total_profit
from dealers_dow_combination as a
left join dealer_sales as b on
    a.dealer_name = b.dealer_name
    and a.date_dow = b.date_dow
order by a.dealer_name asc, a.date_dow asc
"""
).df()
dealers_gen_calc_2

,dealer_name,date_dow,date_dow_name,total_units_sold,total_sales,total_profit
0,AutoHub Cars,0,Sunday,15,308000.0,42200.0
1,AutoHub Cars,1,Monday,8,168000.0,19000.0
2,AutoHub Cars,2,Tuesday,6,144000.0,16600.0
3,AutoHub Cars,3,Wednesday,0,0.0,0.0
4,AutoHub Cars,4,Thursday,0,0.0,0.0
...,...,...,...,...,...,...
65,Visayas Vehicles,2,Tuesday,12,306000.0,30600.0
66,Visayas Vehicles,3,Wednesday,0,0.0,0.0
67,Visayas Vehicles,4,Thursday,0,0.0,0.0
68,Visayas Vehicles,5,Friday,14,256000.0,28800.0


**dealers_gen_calc_3**


calculation of total sales, profit, units sold per Date

In [12]:
"""
Dealers General Calculation 3:
Daily Sales per dealer
"""

dealers_gen_calc_3 = db.sql(
    """
    select
        date,
        sum(qty_sold) as total_units_sold,
        sum(total_price_usd) as total_sales,
        sum(total_profit_usd) as total_profit
    from df
    group by 1
    order by date asc
    """
).df()
dealers_gen_calc_3

,date,total_units_sold,total_sales,total_profit
0,2024-01-05,1.0,25000.0,2500.0
1,2024-01-08,1.0,28000.0,2800.0
2,2024-02-12,2.0,28000.0,2800.0
3,2024-02-22,1.0,20000.0,2000.0
4,2024-03-01,3.0,96000.0,9600.0
...,...,...,...,...
213,2024-12-23,1.0,24000.0,2400.0
214,2024-12-24,2.0,48000.0,4800.0
215,2024-12-27,1.0,14000.0,1400.0
216,2024-12-28,1.0,25000.0,2500.0


**dealers_calc_1**

calculation of total sales, profit, units sold of each brand per dealer

In [13]:
"""
Dealers Calculation 1:
1. Sales, profit, units sold of each brand per dealer

"""

dealers_calc_1 = db.sql(
"""
with brands as (
    select
        distinct brand
    from df
),
    dealers as (
        select
            distinct dealer_name
        from df
),
    dealers_brands as (
        select
            a.dealer_name,
            b.brand
        from dealers as a
        cross join brands as b        
        order by a.dealer_name asc, b.brand asc
),
    dealers_sales_brands as (
        select
            dealer_name,
            brand,
            sum(qty_sold)::int as total_units_sold,
            sum(total_price_usd) as total_sales,
            sum(total_profit_usd) as total_profit
        from df
        group by 1,2
        order by dealer_name asc, brand asc        
)

select
    a.dealer_name,
    a.brand,
    coalesce(b.total_units_sold,0)::int as total_units_sold,
    coalesce(b.total_sales,0) as total_sales,
    coalesce(b.total_profit,0) as total_profit
from dealers_brands as a
left join dealers_sales_brands as b
on a.dealer_name = b.dealer_name
and a.brand = b.brand
order by a.dealer_name asc,  b.total_units_sold desc
"""
).df()
dealers_calc_1

,dealer_name,brand,total_units_sold,total_sales,total_profit
0,AutoHub Cars,Subaru,9,160000.0,16000.0
1,AutoHub Cars,Honda,8,182000.0,33600.0
2,AutoHub Cars,Ford,5,145000.0,14500.0
3,AutoHub Cars,Hyundai,5,116000.0,16400.0
4,AutoHub Cars,Land Rover,5,106000.0,10600.0
...,...,...,...,...,...
75,Visayas Vehicles,Hyundai,5,116000.0,16400.0
76,Visayas Vehicles,Toyota,5,99000.0,17400.0
77,Visayas Vehicles,Mazda,5,120000.0,12000.0
78,Visayas Vehicles,Mitsubishi,4,100000.0,10000.0


**dealers_calc_2**

calculation of total sales, units sold, profit of each brand and model for each dealer

In [14]:
dealers_calc_2 = db.sql(
"""
with brands_models as (
    select
        distinct brand,
        model,
        segment,
        engine_size_l,
        fuel_type,
        unit_price_usd,
        unit_profit_usd
    from df
    order by brand asc
),
    dealers as (
    select
        distinct dealer_name
    from df       
),
    brands_models_dealers as (
    select
        a.dealer_name,
        b.brand,
        b.model,
        b.segment,
        b.engine_size_l,
        b.fuel_type,
        b.unit_price_usd,
        b.unit_profit_usd
    from dealers as a
    cross join brands_models as b
    order by a.dealer_name asc, b.brand asc
),
    sales as (
    select
        dealer_name,
        brand,
        model,
        segment,
        engine_size_l,
        fuel_type,
        unit_price_usd,
        unit_profit_usd,
        sum(qty_sold)::int as total_units_sold,
        sum(total_price_usd) as total_sales,
        sum(total_profit_usd) as total_profit
    from df
    group by 1,2,3,4,5,6,7,8
    order by dealer_name asc, brand asc

)
    select
        a.dealer_name,
        a.brand,
        a.model,
        a.segment,
        a.engine_size_l,
        a.fuel_type,
        a.unit_price_usd,
        a.unit_profit_usd,
        coalesce(b.total_units_sold,0)::int as total_units_sold,
        coalesce(b.total_sales,0) as total_sales,
        coalesce(b.total_profit,0) as total_profit
    from brands_models_dealers as a
    left join sales as b
        on a.dealer_name = b.dealer_name
        and a.brand = b.brand
        and a.model = b.model
        and a.segment = b.segment
        and a.engine_size_l = b.engine_size_l
        and a.fuel_type = b.fuel_type
        and a.unit_price_usd = b.unit_price_usd
        and a.unit_profit_usd = b.unit_profit_usd
    order by a.dealer_name asc, a.brand asc

"""
).df()
dealers_calc_2


,dealer_name,brand,model,segment,engine_size_l,fuel_type,unit_price_usd,unit_profit_usd,total_units_sold,total_sales,total_profit
0,AutoHub Cars,Ford,Everest,SUV,2.0,Diesel,30000.0,3000.0,4,120000.0,12000.0
1,AutoHub Cars,Ford,Ranger,Pickup,2.0,Diesel,25000.0,2500.0,1,25000.0,2500.0
2,AutoHub Cars,Honda,Civic,Sedan,1.8,Gasoline,22000.0,4400.0,7,154000.0,30800.0
3,AutoHub Cars,Honda,CR-V,SUV,2.4,Gasoline,28000.0,2800.0,1,28000.0,2800.0
4,AutoHub Cars,Hyundai,Accent,Sedan,1.6,Gasoline,16000.0,3200.0,3,48000.0,9600.0
...,...,...,...,...,...,...,...,...,...,...,...
145,Visayas Vehicles,Mitsubishi,Xpander,MPV,1.5,Gasoline,18000.0,1800.0,2,36000.0,3600.0
146,Visayas Vehicles,Subaru,Impreza,Sedan,1.5,Gasoline,15000.0,1500.0,3,45000.0,4500.0
147,Visayas Vehicles,Subaru,Forester,SUV,1.8,Gasoline,20000.0,2000.0,8,160000.0,16000.0
148,Visayas Vehicles,Toyota,Corolla Cross,SUV,1.8,Gasoline,25000.0,5000.0,3,75000.0,15000.0


### **Geographical Calculations**

In [15]:
"""
Geographical Calculation 1:

Total sales, units sold, profit per country and city

"""

geo_calc_1 = db.sql(
"""
select
    dealer_country as country,
    dealer_city as city,
    sum(qty_sold)::int as total_units_sold,
    sum(total_price_usd) as total_sales,
    sum(total_profit_usd) as total_profit
from df
group by 1,2
order by country asc
"""
).df()
geo_calc_1

,country,city,total_units_sold,total_sales,total_profit
0,Malaysia,George Town,42,925000.0,112700.0
1,Malaysia,Kuala Lumpur,28,608000.0,75000.0
2,Philippines,Bacolod City,22,522000.0,56000.0
3,Philippines,Cebu City,46,980000.0,116900.0
4,Philippines,Manila,44,1003000.0,119400.0
5,Philippines,Iloilo City,36,827000.0,89100.0
6,Philippines,Quezon City,43,1014000.0,113800.0
7,Philippines,Davao City,30,679000.0,78100.0
8,Singapore,Singapore,28,672000.0,77200.0
9,Thailand,Bangkok,28,661000.0,74300.0


In [32]:
"""
Geographical Calculation 2:

Total sales, units sold, profit per country and city by month.
This query is a modified query from dealers_gen_calc_1 which calculates monthly sales for all dealers.

"""


geo_calc_2 = db.sql(
"""
with months as (
    select
        unnest([1,2,3,4,5,6,7,8,9,10,11,12]) as date_month,
        unnest([
            'January','February','March','April',
            'May','June','July','August',
            'September','October','November','December'
        ]) as date_month_name
),
dealers as (
    select
        distinct(dealer_name) as dealer_name,
        dealer_country as country,
        dealer_city as city
    from df
),
 dealer_months_combination as (
     select
        a.dealer_name,
        a.country,
        a.city,
        b.date_month,
        b.date_month_name
     from dealers as a 
     cross join months as b
     order by a.dealer_name asc, b.date_month asc
 ),
dealer_sales as (
    select
        dealer_name,
        dealer_country as country,
        dealer_city as city,
        date_month,
        date_month_name,
        sum(qty_sold) as total_units_sold,
        sum(total_price_usd) as total_sales,
        sum(total_profit_usd) as total_profit
    from df
    group by 1,2,3,4,5
)

select
    a.dealer_name,
    a.country,
    a.city,
    a.date_month,
    a.date_month_name,
    coalesce(b.total_units_sold, 0)::int as total_units_sold,
    coalesce(b.total_sales, 0)::float as total_sales,
    coalesce(b.total_profit, 0)::float as total_profit
from dealer_months_combination as a
left join dealer_sales as b on
    a.dealer_name = b.dealer_name
    and a.country = b.country
    and a.city = b.city
    and a.date_month = b.date_month
order by a.dealer_name asc, a.date_month asc

"""
).df()
geo_calc_2

,dealer_name,country,city,date_month,date_month_name,total_units_sold,total_sales,total_profit
0,AutoHub Cars,Malaysia,George Town,1,January,0,0.0,0.0
1,AutoHub Cars,Malaysia,George Town,2,February,0,0.0,0.0
2,AutoHub Cars,Malaysia,George Town,3,March,4,76000.0,7600.0
3,AutoHub Cars,Malaysia,George Town,4,April,6,124000.0,17200.0
4,AutoHub Cars,Malaysia,George Town,5,May,1,34000.0,3400.0
...,...,...,...,...,...,...,...,...
115,Visayas Vehicles,Philippines,Cebu City,8,August,7,186000.0,18600.0
116,Visayas Vehicles,Philippines,Cebu City,9,September,6,141000.0,28200.0
117,Visayas Vehicles,Philippines,Cebu City,10,October,4,74000.0,7400.0
118,Visayas Vehicles,Philippines,Cebu City,11,November,3,64000.0,6400.0


In [17]:
"""
Geographical Calculation 3:

Total sales, units sold, profit per country and city by day of week.
This query is a modified query from dealers_gen_calc_2 which calculates dow sales for all dealers.

"""

geo_calc_3 = db.sql(
"""
with dow_names as (
    select
        unnest([0,1,2,3,4,5,6]) as date_dow,
        unnest([
            'Sunday','Monday','Tuesday','Wednesday',
            'Thursday','Friday','Saturday'
        ]) as date_dow_name
),
dealers as (
    select
        distinct dealer_name as dealer_name,
        dealer_country as country,
        dealer_city as city
    from df
),
 dealers_dow_combination as (
     select
        a.dealer_name,
        a.country,
        a.city,
        b.date_dow,
        b.date_dow_name
     from dealers as a 
     cross join dow_names as b
     order by a.dealer_name asc, a.country asc, b.date_dow asc
 ),
dealer_sales as (
    select
        dealer_name,
        dealer_country,
        dealer_city,
        date_dow,
        date_dow_name,
        sum(qty_sold) as total_units_sold,
        sum(total_price_usd) as total_sales,
        sum(total_profit_usd) as total_profit
    from df
    group by 1,2,3,4,5
)

select
    a.dealer_name,
    a.country,
    a.city,
    a.date_dow,
    a.date_dow_name,
    coalesce(b.total_units_sold, 0)::int as total_units_sold,
    coalesce(b.total_sales, 0)::float as total_sales,
    coalesce(b.total_profit, 0)::float as total_profit
from dealers_dow_combination as a
left join dealer_sales as b on
    a.dealer_name = b.dealer_name
    and a.date_dow = b.date_dow
order by a.dealer_name asc, a.date_dow asc

"""
).df()
geo_calc_3

,dealer_name,country,city,date_dow,date_dow_name,total_units_sold,total_sales,total_profit
0,AutoHub Cars,Malaysia,George Town,0,Sunday,15,308000.0,42200.0
1,AutoHub Cars,Malaysia,George Town,1,Monday,8,168000.0,19000.0
2,AutoHub Cars,Malaysia,George Town,2,Tuesday,6,144000.0,16600.0
3,AutoHub Cars,Malaysia,George Town,3,Wednesday,0,0.0,0.0
4,AutoHub Cars,Malaysia,George Town,4,Thursday,0,0.0,0.0
...,...,...,...,...,...,...,...,...
65,Visayas Vehicles,Philippines,Cebu City,2,Tuesday,12,306000.0,30600.0
66,Visayas Vehicles,Philippines,Cebu City,3,Wednesday,0,0.0,0.0
67,Visayas Vehicles,Philippines,Cebu City,4,Thursday,0,0.0,0.0
68,Visayas Vehicles,Philippines,Cebu City,5,Friday,14,256000.0,28800.0


In [6]:
"""
Geographical Calculation 4:

Total sales, profit, units sold for each brand,model,segment by country,city
"""

geo_calc_4 = db.sql(
"""
-- dim_facts_records.car_sales_records
with brands_models as (
    select
        distinct brand,
        model,
        segment,
        engine_size_l,
        fuel_type,
        unit_price_usd,
        unit_profit_usd
    from df
    order by brand asc
),
    dealers as (
    select
        distinct dealer_name,
        dealer_country as country,
        dealer_city as city
    from df       
),
    brands_models_dealers as (
    select
        a.dealer_name,
        a.country,
        a.city,
        b.brand,
        b.model,
        b.segment,
        b.engine_size_l,
        b.fuel_type,
        b.unit_price_usd,
        b.unit_profit_usd
    from dealers as a
    cross join brands_models as b
    order by a.dealer_name asc, b.brand asc
),
    sales as (
    select
        dealer_name,
        dealer_country,
        dealer_city,
        brand,
        model,
        segment,
        engine_size_l,
        fuel_type,
        unit_price_usd,
        unit_profit_usd,
        sum(qty_sold)::int as total_units_sold,
        sum(total_price_usd) as total_sales,
        sum(total_profit_usd) as total_profit
    from df
    group by 1,2,3,4,5,6,7,8,9,10
    order by dealer_name asc, brand asc

)
    select
        a.dealer_name,
        a.country,
        a.city,
        a.brand,
        a.model,
        a.segment,
        a.engine_size_l,
        a.fuel_type,
        a.unit_price_usd,
        a.unit_profit_usd,
        coalesce(b.total_units_sold,0)::int as total_units_sold,
        coalesce(b.total_sales,0) as total_sales,
        coalesce(b.total_profit,0) as total_profit
    from brands_models_dealers as a
    left join sales as b
        on a.dealer_name = b.dealer_name
        and a.country = b.dealer_country
        and a.city = b.dealer_city
        and a.brand = b.brand
        and a.model = b.model
        and a.segment = b.segment
        and a.engine_size_l = b.engine_size_l
        and a.fuel_type = b.fuel_type
        and a.unit_price_usd = b.unit_price_usd
        and a.unit_profit_usd = b.unit_profit_usd
    order by a.dealer_name asc, a.brand asc
"""
).df()
geo_calc_4

,dealer_name,country,city,brand,model,segment,engine_size_l,fuel_type,unit_price_usd,unit_profit_usd,total_units_sold,total_sales,total_profit
0,AutoHub Cars,Malaysia,George Town,Ford,Ranger,Pickup,2.0,Diesel,25000.0,2500.0,1,25000.0,2500.0
1,AutoHub Cars,Malaysia,George Town,Ford,Everest,SUV,2.0,Diesel,30000.0,3000.0,4,120000.0,12000.0
2,AutoHub Cars,Malaysia,George Town,Honda,Civic,Sedan,1.8,Gasoline,22000.0,4400.0,7,154000.0,30800.0
3,AutoHub Cars,Malaysia,George Town,Honda,CR-V,SUV,2.4,Gasoline,28000.0,2800.0,1,28000.0,2800.0
4,AutoHub Cars,Malaysia,George Town,Hyundai,Accent,Sedan,1.6,Gasoline,16000.0,3200.0,3,48000.0,9600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Visayas Vehicles,Philippines,Cebu City,Mitsubishi,Outlander,SUV,2.4,Gasoline,32000.0,3200.0,2,64000.0,6400.0
146,Visayas Vehicles,Philippines,Cebu City,Subaru,Impreza,Sedan,1.5,Gasoline,15000.0,1500.0,3,45000.0,4500.0
147,Visayas Vehicles,Philippines,Cebu City,Subaru,Forester,SUV,1.8,Gasoline,20000.0,2000.0,8,160000.0,16000.0
148,Visayas Vehicles,Philippines,Cebu City,Toyota,Vios,Sedan,1.5,Gasoline,12000.0,1200.0,2,24000.0,2400.0


### **Brands, Models Calculations**

In [ ]:
"""
Products Calculation 1:

Monthly Total sales, units sold, profit of all brands and its details

"""


products_calc_1 = db.sql(
"""
with months as (
    select
        unnest([1,2,3,4,5,6,7,8,9,10,11,12]) as date_month,
        unnest([
            'January','February','March','April',
            'May','June','July','August',
            'September','October','November','December'
        ]) as date_month_name
),
products as (
    select
        distinct brand,
        model,
        segment,
        engine_size_l,
        fuel_type,
        unit_price_usd,
        unit_profit_usd
    from df
    order by brand asc
),
 products_months_combination as (
     select
        a.brand,
        a.model,
        a.segment,
        a.engine_size_l,
        a.fuel_type,
        a.unit_price_usd,
        a.unit_profit_usd,
        b.date_month,
        b.date_month_name
     from products as a 
     cross join months as b
     order by a.brand asc, b.date_month asc
 ),
product_sales as (
    select
        brand,
        model,
        segment,
        engine_size_l,
        fuel_type,
        unit_price_usd,
        unit_profit_usd,
        date_month,
        date_month_name,        
        sum(qty_sold) as total_units_sold,
        sum(total_price_usd) as total_sales,
        sum(total_profit_usd) as total_profit
    from df
    group by 1,2,3,4,5,6,7,8,9
)

select
    a.brand,
    a.model,
    a.segment,
    a.engine_size_l,
    a.fuel_type,
    a.unit_price_usd,
    a.unit_profit_usd,
    a.date_month,
    a.date_month_name,
    coalesce(b.total_units_sold, 0)::int as total_units_sold,
    coalesce(b.total_sales, 0)::float as total_sales,
    coalesce(b.total_profit, 0)::float as total_profit
from products_months_combination as a
left join product_sales as b on
    a.brand = b.brand
    and a.model = b.model
    and a.segment = b.segment
    and a.engine_size_l = b.engine_size_l
    and a.fuel_type = b.fuel_type
    and a.unit_price_usd = b.unit_price_usd
    and a.unit_profit_usd = b.unit_profit_usd
    and a.date_month = b.date_month
order by a.brand asc, a.model asc, a.date_month asc

"""
).df()
products_calc_1

,brand,model,segment,engine_size_l,fuel_type,unit_price_usd,unit_profit_usd,date_month,date_month_name,total_units_sold,total_sales,total_profit
0,Ford,Everest,SUV,2.0,Diesel,30000.0,3000.0,1,January,0,0.0,0.0
1,Ford,Everest,SUV,2.0,Diesel,30000.0,3000.0,2,February,0,0.0,0.0
2,Ford,Everest,SUV,2.0,Diesel,30000.0,3000.0,3,March,1,30000.0,3000.0
3,Ford,Everest,SUV,2.0,Diesel,30000.0,3000.0,4,April,2,60000.0,6000.0
4,Ford,Everest,SUV,2.0,Diesel,30000.0,3000.0,5,May,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
175,Toyota,Vios,Sedan,1.5,Gasoline,12000.0,1200.0,8,August,8,96000.0,9600.0
176,Toyota,Vios,Sedan,1.5,Gasoline,12000.0,1200.0,9,September,0,0.0,0.0
177,Toyota,Vios,Sedan,1.5,Gasoline,12000.0,1200.0,10,October,3,36000.0,3600.0
178,Toyota,Vios,Sedan,1.5,Gasoline,12000.0,1200.0,11,November,0,0.0,0.0


In [22]:
products_calc_2 = db.sql(
"""
with dow as (
    select
        unnest([0,1,2,3,4,5,6]) as date_dow,
        unnest([
            'Sunday','Monday','Tuesday','Wednesday',
            'Thursday','Friday','Saturday'
        ]) as date_dow_name
),
products as (
    select
        distinct brand,
        model,
        segment,
        engine_size_l,
        fuel_type,
        unit_price_usd,
        unit_profit_usd
    from df
    order by brand asc
),
 products_months_combination as (
     select
        a.brand,
        a.model,
        a.segment,
        a.engine_size_l,
        a.fuel_type,
        a.unit_price_usd,
        a.unit_profit_usd,
        b.date_dow,
        b.date_dow_name
     from products as a 
     cross join dow as b
     order by a.brand asc, b.date_dow asc
 ),
product_sales as (
    select
        brand,
        model,
        segment,
        engine_size_l,
        fuel_type,
        unit_price_usd,
        unit_profit_usd,
        date_dow,
        date_dow_name,        
        sum(qty_sold) as total_units_sold,
        sum(total_price_usd) as total_sales,
        sum(total_profit_usd) as total_profit
    from df
    group by 1,2,3,4,5,6,7,8,9
)

select
    a.brand,
    a.model,
    a.segment,
    a.engine_size_l,
    a.fuel_type,
    a.unit_price_usd,
    a.unit_profit_usd,
    a.date_dow,
    a.date_dow_name,
    coalesce(b.total_units_sold, 0)::int as total_units_sold,
    coalesce(b.total_sales, 0)::float as total_sales,
    coalesce(b.total_profit, 0)::float as total_profit
from products_months_combination as a
left join product_sales as b on
    a.brand = b.brand
    and a.date_dow = b.date_dow
order by a.brand asc, a.date_dow asc

"""
).df()
products_calc_2

,brand,model,segment,engine_size_l,fuel_type,unit_price_usd,unit_profit_usd,date_dow,date_dow_name,total_units_sold,total_sales,total_profit
0,Ford,Ranger,Pickup,2.0,Diesel,25000.0,2500.0,0,Sunday,9,225000.0,22500.0
1,Ford,Everest,SUV,2.0,Diesel,30000.0,3000.0,0,Sunday,9,225000.0,22500.0
2,Ford,Ranger,Pickup,2.0,Diesel,25000.0,2500.0,0,Sunday,3,90000.0,9000.0
3,Ford,Everest,SUV,2.0,Diesel,30000.0,3000.0,0,Sunday,3,90000.0,9000.0
4,Ford,Ranger,Pickup,2.0,Diesel,25000.0,2500.0,1,Monday,3,90000.0,9000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
162,Toyota,Corolla Cross,SUV,1.8,Gasoline,25000.0,5000.0,4,Thursday,0,0.0,0.0
163,Toyota,Vios,Sedan,1.5,Gasoline,12000.0,1200.0,5,Friday,5,60000.0,6000.0
164,Toyota,Corolla Cross,SUV,1.8,Gasoline,25000.0,5000.0,5,Friday,5,60000.0,6000.0
165,Toyota,Vios,Sedan,1.5,Gasoline,12000.0,1200.0,6,Saturday,3,36000.0,3600.0


In [23]:
products_calc_3 = db.sql(
"""
select
    brand,
    model,
    segment,
    engine_size_l,
    fuel_type,
    unit_price_usd,
    unit_profit_usd,        
    sum(qty_sold) as total_units_sold,
    sum(total_price_usd) as total_sales,
    sum(total_profit_usd) as total_profit
from df
group by 1,2,3,4,5,6,7
order by brand asc

"""
).df()
products_calc_3

,brand,model,segment,engine_size_l,fuel_type,unit_price_usd,unit_profit_usd,total_units_sold,total_sales,total_profit
0,Ford,Ranger,Pickup,2.0,Diesel,25000.0,2500.0,23.0,575000.0,57500.0
1,Ford,Everest,SUV,2.0,Diesel,30000.0,3000.0,20.0,600000.0,60000.0
2,Honda,CR-V,SUV,2.4,Gasoline,28000.0,2800.0,37.0,1036000.0,103600.0
3,Honda,Civic,Sedan,1.8,Gasoline,22000.0,4400.0,23.0,506000.0,101200.0
4,Hyundai,Santa Fe,SUV,2.4,Gasoline,34000.0,3400.0,35.0,1190000.0,119000.0
5,Hyundai,Accent,Sedan,1.6,Gasoline,16000.0,3200.0,33.0,528000.0,105600.0
6,Land Rover,Range Rover,SUV,3.0,Gasoline,26000.0,2600.0,13.0,338000.0,33800.0
7,Land Rover,Defender,SUV,1.8,Gasoline,14000.0,1400.0,18.0,252000.0,25200.0
8,Mazda,Mazda3,Sedan,2.0,Gasoline,24000.0,2400.0,26.0,624000.0,62400.0
9,Mitsubishi,Outlander,SUV,2.4,Gasoline,32000.0,3200.0,16.0,512000.0,51200.0


In [5]:
products_calc_4 = db.sql(
"""
with brands as (
    select
        distinct brand
    from df
),
    countries as (
    select
        distinct dealer_country
    from df        
),
    brandsxcountries as (
    select
        a.brand,
        b.dealer_country
    from brands as a
    cross join countries as b
),
    brands_calculations as (
    select
        brand,
        dealer_country,
        sum(qty_sold)::int as total_units_sold,
        sum(total_price_usd) as total_sales,
        sum(total_profit_usd) as total_profit
    from df
    group by 1,2
    order by brand asc, dealer_country asc
)

select
    a.brand,
    a.dealer_country,
    coalesce(b.total_units_sold,0) as total_untis_sold,
    coalesce(b.total_sales,0) as total_sales,
    coalesce(b.total_profit,0) as total_profit
from brandsxcountries as a
left join brands_calculations as b
on a.brand = b.brand
and a.dealer_country = b.dealer_country
order by a.brand asc, a.dealer_country asc

"""
).df()
products_calc_4

,brand,dealer_country,total_untis_sold,total_sales,total_profit
0,Ford,Malaysia,8,230000.0,23000.0
1,Ford,Philippines,28,760000.0,76000.0
2,Ford,Singapore,1,30000.0,3000.0
3,Ford,Thailand,6,155000.0,15500.0
4,Honda,Malaysia,14,320000.0,58400.0
5,Honda,Philippines,33,876000.0,105200.0
6,Honda,Singapore,8,224000.0,22400.0
7,Honda,Thailand,5,122000.0,18800.0
8,Hyundai,Malaysia,8,182000.0,26200.0
9,Hyundai,Philippines,51,1248000.0,168000.0
